In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf

from string import punctuation
from collections import Counter
import re
import os

In [2]:
reviews_path = 'C:/Users/p0ng5/OneDrive/movie_data.csv'

In [3]:
#reviews_path = 'C:/Users/LENOVO/Desktop/movie_data.csv'

In [4]:
movie_data = pd.read_csv(reviews_path)

In [5]:
movie_data.head()

,review,sentiment
0,Nothing is fantastic! Simple as that! It's a f...,1
1,This really was a waste of time...the movie ha...,0
2,"This might sound weird, but I only got to see ...",0
3,Considering this film was released 8 years bef...,1
4,Very rarely does one come across an indie come...,1


In [6]:
movie_data.tail()

,review,sentiment
49995,"I Sell the Dead is a big, sloppy horror comedy...",1
49996,I know this sounds odd coming from someone bor...,1
49997,OK I had higher hopes for this Carnosaur movie...,0
49998,"Laurence Fishburne is a fine actor, and deserv...",1
49999,I lived in Tokyo for 7 months. Knowing the rea...,1


In [7]:
# count unique words
counter = Counter()

# clean text
for i, review in enumerate(movie_data['review']):
    
    text = "".join(c if c not in punctuation else " {} ".format(c) for c in review).lower()
    movie_data.iloc[i, 0] = text
    
    counter.update(text.split())

In [8]:
# word2int dictionary
word2int = {w:i for i, w in enumerate(sorted(counter, key=counter.get, reverse=True), start=1)}

In [9]:
# int-list reviews
mapped_reviews = []
for review in movie_data['review']:
    mapped_reviews.append([word2int[word] for word in review.split()])

In [10]:
sequence_length = 200

# if length < sequence_length : left padd with zeros
# if length > sequence_length : take last 'sequence_length' elements

# padded sequence
sequences = np.zeros(shape=(len(movie_data), sequence_length), dtype=int)

In [11]:
for i, mapped_review in enumerate(mapped_reviews):
    n = len(mapped_review)
    if n < sequence_length:
        sequences[i, -n:] = mapped_review
    else:
        sequences[i, :] = mapped_review[-sequence_length:]

In [12]:
X_train = sequences[:25000, :]
y_train = movie_data.iloc[:25000, 1].values

X_test = sequences[25000:, :]
y_test = movie_data.iloc[25000:, 1].values

X_train.shape, y_train.shape, X_test.shape, y_train.shape

((25000, 200), (25000,), (25000, 200), (25000,))

In [13]:
X_val = X_test[:12500]
y_val = y_test[:12500]
X_test = X_test[12500:]
y_test = y_test[12500:]

X_val.shape, y_val.shape, X_test.shape, y_test.shape

((12500, 200), (12500,), (12500, 200), (12500,))

In [14]:
def batch_generator(X, y=None, batch_size=64):
    if y is not None:
        assert len(X) == len(y)
    n_batch = len(X) // batch_size
    for i in range(n_batch):
        a = i*batch_size
        b = (i+1)*batch_size
        
        if y is not None:
            yield X[a:b], y[a:b]
        else:
            yield X[a:b]

In [2]:
class SentimentRNN:
    
    def __init__(self, word_size, embed_size=200, lstm_size=256, num_layer=1,
                seq_length=200, learning_rate=1e-4, batch_size=32):
        # model hyper parameters
        self.word_size = word_size
        self.embed_size = embed_size
        self.lstm_size = lstm_size
        self.num_layer = num_layer
        self.seq_length = seq_length
        self.learning_rate = learning_rate
        self.batch_size = batch_size
        
        # build model graph
        self.g = tf.Graph()
        with self.g.as_default():
            #tf.set_random_seed(123)
            self.build()
            self.saver = tf.train.Saver()
            self.init_op = tf.global_variables_initializer()
    
    def build(self):
        # placeholder for inputs
        tf_x = tf.placeholder(dtype=tf.int32, 
                              shape=(self.batch_size, self.seq_length), 
                              name='tf_x')
        tf_y = tf.placeholder(dtype=tf.float32, 
                              shape=(self.batch_size), 
                              name='tf_y')
        tf_keepprob = tf.placeholder(dtype=tf.float32, shape=(), 
                                     name='tf_keepprob')
        # embedding vector
        W_embedding = tf.Variable(
            tf.random_uniform(shape=(self.word_size, self.embed_size), minval=-1, maxval=1),
            name='W_embedding')
        
        embed_x = tf.nn.embedding_lookup(W_embedding, tf_x, name='embed_x')
        
        # create rnn cell
        cells = tf.contrib.rnn.MultiRNNCell([
            tf.contrib.rnn.DropoutWrapper(tf.contrib.rnn.BasicLSTMCell(self.lstm_size),
                                         output_keep_prob=tf_keepprob)
                                          for i in range(self.num_layer)
        ])
        
        # define the initial state/ rnn steps
        self.initial_state = cells.zero_state(self.batch_size, tf.float32)
        print("  << initial state > ", self.initial_state)
        
        lstm_outputs, self.final_state = tf.nn.dynamic_rnn(cell=cells, inputs=embed_x, 
                                                           initial_state=self.initial_state)
        print("\n << lstm_output >> ", lstm_outputs)
        print("\n << final state >> ", self.final_state)
        
        # dense layer -> logits
        logits = tf.layers.dense(inputs=lstm_outputs[:, -1], units=1, 
                                 activation=None, name='logits')
        
        logits = tf.squeeze(logits, name='logits_squeezed')
        print('\n  << logits     >> ', logits)
        
        # predictions -> prob. | labels
        y_proba = tf.nn.sigmoid(logits, name='probabilities')
        y_labels = tf.cast(tf.round(y_proba), dtype=tf.int32, name='labels')
        predictions = {
            'probabilities': y_proba,
            'labels': y_labels
        }
        print("\n << predictions  >> ", predictions)
        
        # cost function
        cost = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(labels=tf_y, 
                                                                      logits=logits), 
                              name='cost')
        # optimizer
        optimizer = tf.train.AdamOptimizer(self.learning_rate)
        train_op = optimizer.minimize(cost, name='train_op')
    
    def train(self, X, y, num_epochs):
        with tf.Session(graph=self.g) as sess:
            sess.run(self.init_op)
            
            batch_total = 0
            for epoch in range(num_epochs):
                # reset cell&hidden states
                state = sess.run(self.initial_state)
                for x_batch, y_batch in batch_generator(X, y, batch_size=self.batch_size):
                    feed = {'tf_x:0': x_batch,
                            'tf_y:0': y_batch,
                            'tf_keepprob:0': 0.5,
                            self.initial_state: state}
                    loss, _, state = sess.run(['cost:0', 'train_op', self.final_state], 
                                              feed_dict=feed)
                    
                    # update training every 20 batches
                    batch_total += 1
                    if (batch_total+1) % 20 == 0:
                        print("Epoch {:3d}, Iterations {:4d} | Train loss: {:.4f}".format(epoch+1, 
                                                                                            batch_total+1,
                                                                                            loss))
                # save every 10 epochs                                                                          
                if (epoch + 1) % 10 == 0:
                    self.saver.save(sess, 
                                    'model/sentiment-rnn-{}.ckpt'.format(epoch+1))
    
    def predict(self, X, predict_proba=False):
        pred = []
        with tf.Session(graph=self.g) as sess:
            # restore latest model
            self.saver.restore(sess, 
                               tf.train.latest_checkpoint('./model/'))
            
            # init model states
            state = sess.run(self.initial_state)
            
            for x_batch in batch_generator(X, y=None, batch_size=self.batch_size):
                feed = {'tf_x:0': x_batch,
                        'tf_keepprob:0': 1.0,
                        self.initial_state: state}
                
                if predict_proba:
                    y_pred, state = sess.run(['probabilities:0', self.final_state], 
                                           feed_dict=feed)
                else:
                    y_pred, state = sess.run(['labels:0', self.final_state], 
                                            feed_dict=feed)
                pred.append(y_pred)
                
        return np.concatenate(pred)

In [3]:
sequence_length = 200
word_size = max(word2int.values()) + 1
embed_size = 256
hidden_size = 128
n_layer = 1
batch_size = 100
learning_rate = 0.001

model = SentimentRNN(word_size=word_size, embed_size=embed_size, lstm_size=hidden_size, 
                     num_layer=n_layer, seq_length=sequence_length, batch_size=batch_size, 
                     learning_rate=learning_rate)


NameError: name 'word2int' is not defined

In [24]:
model.train(X_train, y_train, num_epochs=40)

Epoch   1, Iterations   20 | Train loss: 0.6689
Epoch   1, Iterations   40 | Train loss: 0.5714
Epoch   1, Iterations   60 | Train loss: 0.6164
Epoch   1, Iterations   80 | Train loss: 0.6584
Epoch   1, Iterations  100 | Train loss: 0.5446
Epoch   1, Iterations  120 | Train loss: 0.4987
Epoch   1, Iterations  140 | Train loss: 0.4725
Epoch   1, Iterations  160 | Train loss: 0.4996
Epoch   1, Iterations  180 | Train loss: 0.5622
Epoch   1, Iterations  200 | Train loss: 0.5063
Epoch   1, Iterations  220 | Train loss: 0.4509
Epoch   1, Iterations  240 | Train loss: 0.4027
Epoch   2, Iterations  260 | Train loss: 0.5039
Epoch   2, Iterations  280 | Train loss: 0.3833
Epoch   2, Iterations  300 | Train loss: 0.4192
Epoch   2, Iterations  320 | Train loss: 0.4525
Epoch   2, Iterations  340 | Train loss: 0.3851
Epoch   2, Iterations  360 | Train loss: 0.2664
Epoch   2, Iterations  380 | Train loss: 0.2420
Epoch   2, Iterations  400 | Train loss: 0.3306
Epoch   2, Iterations  420 | Train loss:

Epoch  14, Iterations 3440 | Train loss: 0.0023
Epoch  14, Iterations 3460 | Train loss: 0.0097
Epoch  14, Iterations 3480 | Train loss: 0.0064
Epoch  14, Iterations 3500 | Train loss: 0.0044
Epoch  15, Iterations 3520 | Train loss: 0.0028
Epoch  15, Iterations 3540 | Train loss: 0.0016
Epoch  15, Iterations 3560 | Train loss: 0.0060
Epoch  15, Iterations 3580 | Train loss: 0.0148
Epoch  15, Iterations 3600 | Train loss: 0.0145
Epoch  15, Iterations 3620 | Train loss: 0.0009
Epoch  15, Iterations 3640 | Train loss: 0.0011
Epoch  15, Iterations 3660 | Train loss: 0.0178
Epoch  15, Iterations 3680 | Train loss: 0.0008
Epoch  15, Iterations 3700 | Train loss: 0.0006
Epoch  15, Iterations 3720 | Train loss: 0.0062
Epoch  15, Iterations 3740 | Train loss: 0.0729
Epoch  16, Iterations 3760 | Train loss: 0.0598
Epoch  16, Iterations 3780 | Train loss: 0.0007
Epoch  16, Iterations 3800 | Train loss: 0.0042
Epoch  16, Iterations 3820 | Train loss: 0.0054
Epoch  16, Iterations 3840 | Train loss:

Epoch  28, Iterations 6860 | Train loss: 0.0004
Epoch  28, Iterations 6880 | Train loss: 0.1193
Epoch  28, Iterations 6900 | Train loss: 0.0111
Epoch  28, Iterations 6920 | Train loss: 0.0125
Epoch  28, Iterations 6940 | Train loss: 0.0076
Epoch  28, Iterations 6960 | Train loss: 0.1508
Epoch  28, Iterations 6980 | Train loss: 0.0128
Epoch  28, Iterations 7000 | Train loss: 0.0163
Epoch  29, Iterations 7020 | Train loss: 0.0052
Epoch  29, Iterations 7040 | Train loss: 0.0028
Epoch  29, Iterations 7060 | Train loss: 0.0023
Epoch  29, Iterations 7080 | Train loss: 0.0222
Epoch  29, Iterations 7100 | Train loss: 0.0033
Epoch  29, Iterations 7120 | Train loss: 0.0136
Epoch  29, Iterations 7140 | Train loss: 0.0175
Epoch  29, Iterations 7160 | Train loss: 0.0210
Epoch  29, Iterations 7180 | Train loss: 0.0033
Epoch  29, Iterations 7200 | Train loss: 0.0223
Epoch  29, Iterations 7220 | Train loss: 0.0043
Epoch  29, Iterations 7240 | Train loss: 0.0011
Epoch  30, Iterations 7260 | Train loss:

In [25]:
# val accuracy
val_preds = model.predict(X_val)

INFO:tensorflow:Restoring parameters from ./model/sentiment-rnn-40.ckpt


In [21]:
y_val

array([1, 1, 1, ..., 0, 1, 1], dtype=int64)

In [ ]:
# tensorboard

In [ ]:
model = SentimentRNN(word_size=word_size, embed_size=embed_size, lstm_size=hidden_size, 
                     num_layer=n_layer, seq_length=sequence_length, batch_size=batch_size, 
                     learning_rate=learning_rate)


### Project 2 - implementing an RNN for character-level language modeling 

In [74]:
with open('pg2265.txt', 'r') as f:
    text = f.read()

In [75]:
len(text)

178707

In [76]:
text = text[15858:]

In [77]:
char_set = set(text)

In [78]:
char2int = {c:i for i,c in enumerate(char_set)}
int2char = {i:c for i,c in enumerate(char_set)}

assert len(char2int) == len(int2char)

In [79]:
text_ints = np.array([char2int[c] for c in text], dtype=np.int32)

In [80]:
text_ints.shape

(162849,)

In [81]:
def reshape_date(sequence, batch_size, num_steps):
    '''generate x, y  sequences from the given sequence
    according to batch_size and num_steps
    
    parameters:
    -----------
    sequence : 1-d array
        a sequence of integers (converted from char2int)
    batch_size : int
        size of batch in each iteration
    num_steps : int
        length of sequence in each batch
    '''
    total_batch_length = batch_size * num_steps
    n_batch = len(sequence) // total_batch_length
    
    # trim original sequence fit to batches
    x = sequence[0:n_batch*total_batch_length]
    y = sequence[1:n_batch*total_batch_length+1]
    assert len(x) == len(y)
    
    # reshape to [batch_size, n_batch*num_steps] matrix
    x = np.asarray(x).reshape(batch_size, n_batch*num_steps)
    y = np.asarray(y).reshape(batch_size, n_batch*num_steps)
    
    return x, y

def batch_generator(data_x, data_y, num_steps):
    batch_size, total_length = data_x.shape
    n_batch = total_length // num_steps
    for i in range(n_batch):
        a, b = i*num_steps, (i+1)*num_steps
        yield data_x[:, a:b], data_y[:, a:b]
                  
    
def get_top_char(proba, char_size, top_n=5):
    p = np.squeeze(proba)
    p[np.argsort(p)[:-top_n]] = 0.0
    p /= np.sum(p)
    idx = np.random.choice(char_size, size=1, p=p)[0]
    return idx

In [82]:
class CharRnn:
    
    def __init__(self, n_classes, lstm_size=128, n_layers=1, 
                 batch_size=64, n_steps=100, learning_rate=1e-3,
                keep_prob=.5, grad_clip=5, sampling=False, random_state=None):
        # model parameters
        self.n_classes = n_classes
        self.lstm_size = lstm_size
        self.batch_size = batch_size
        self.n_steps = n_steps
        self.n_layers = n_layers
        self.learning_rate = learning_rate
        self.keep_prob = keep_prob
        self.grad_clip = grad_clip
        self.random_state = random_state
        
        self.g = tf.Graph()
        with self.g.as_default():
            tf.set_random_seed(self.random_state)
            self.build(sampling)
            self.saver = tf.train.Saver()
            self.init_op = tf.global_variables_initializer()
            
    
    def build(self, sampling):
        if sampling == True:
            # sampling mode
            batch_size, n_step = 1, 1
        else:
            # traing mode
            batch_size, n_step = self.batch_size, self.n_steps
        
        # init placeholders
        tf_x = tf.placeholder(dtype=tf.int32, shape=(batch_size, n_step), name='tf_x')
        tf_y = tf.placeholder(dtype=tf.int32, shape=(batch_size, n_step), name='tf_y')
        tf_keepprob = tf.placeholder(dtype=tf.float32, shape=(), name='tf_keepprob')
        
        # one-hot encoding
        x_onehot = tf.one_hot(tf_x, depth=self.n_classes)
        y_onehot = tf.one_hot(tf_y, depth=self.n_classes)
        print(" << y_onehot >> ", y_onehot)
        # build multi-layers RNN cells
        cells = tf.contrib.rnn.MultiRNNCell([tf.contrib.rnn.DropoutWrapper(
            tf.contrib.rnn.BasicLSTMCell(num_units=self.lstm_size), 
            input_keep_prob=tf_keepprob) for i in range(self.n_layers)])
        
        # define initial state
        self.initial_state = cells.zero_state(batch_size=batch_size, dtype=tf.float32)
        
        # run sequence step through the RNN
        lstm_outputs, self.final_state = tf.nn.dynamic_rnn(cells, 
                                                      inputs=x_onehot, 
                                                      initial_state=self.initial_state)
        print("  << lstm_outputs  >> ", lstm_outputs)
        # reshape output 3-d [batch_size, steps, lstm_size] -> 2-d [batch_size*steps, lstm_size]
        seq_outputs = tf.reshape(lstm_outputs, shape=[-1, self.lstm_size], name='seq_outputs_reshape')
        
        # dense layer -> out shape [seq_length, n_classes]
        logits = tf.layers.dense(seq_outputs, units=self.n_classes, 
                                 activation=None, name='logits')
        print(" << logits >> ", logits)
        proba = tf.nn.softmax(logits, name='probabilities')
        
        y_reshaped = tf.reshape(y_onehot, shape=(-1, self.n_classes), name='y_reshaped')
        
        # cost function
        cost = tf.reduce_mean(
                    tf.nn.softmax_cross_entropy_with_logits(
                                                    labels=y_reshaped, 
                                                    logits=logits), 
                    name='cost')
        
        # gradient clipping to avoiding exploding gradients
        train_vars = tf.trainable_variables()
        grads, _ = tf.clip_by_global_norm(t_list=tf.gradients(cost, train_vars), 
                                          clip_norm=self.grad_clip)
        # optimizer 
        optimizer = tf.train.AdamOptimizer(learning_rate=self.learning_rate)
        train_op = optimizer.apply_gradients(grads_and_vars=zip(grads, train_vars), 
                                             name='train_op')
        
    def train(self, x_train, y_train, n_epochs, ckpt_dir='C:\\Users\\LENOVO\\OneDrive\\tmp\\model'):
        
        # check valid ckpt_dir
        if not os.path.exists(ckpt_dir):
            os.makedirs(ckpt_dir)
        self.ckpt_dir = ckpt_dir
        self.best_loss_ = float('inf')
        with tf.Session(graph=self.g) as sess:
            sess.run(self.init_op)
            
            n_batch = x_train.shape[1]//self.n_steps
            total_iterations = 0
            for i in range(n_epochs):
                # init zero_state
                state = sess.run(self.initial_state)
                avg_loss = 0.0
                for j, (x_batch, y_batch) in enumerate(batch_generator(x_train, y_train, 
                                                                       num_steps=self.n_steps)):
                    feed = {'tf_x:0': x_batch,
                            'tf_y:0': y_batch,
                            'tf_keepprob:0': self.keep_prob, 
                             self.initial_state: state}
                    loss, _, state = sess.run(['cost:0', 'train_op', self.final_state], 
                                              feed_dict=feed)
                    
                    avg_loss += loss            
                    # display every 20 iterations
                    total_iterations += 1
                    if (total_iterations % 20 == 0):
                        print("Epoch {:2d}/{}, Iterations {:4d} | Train loss {:.5f}".format(i, n_epochs, 
                                                                                            total_iterations,
                                                                                            loss))
                # one pass finish
                avg_loss /= (j+1)
                # save model if better than best_loss
                if avg_loss < self.best_loss_:
                    self.best_model_path = os.path.join(self.ckpt_dir, 
                                                       'char_model-{}.ckpt'.format(total_iterations))
                    self.saver.save(sess, self.best_model_path)
    
                        
    def sample(self, output_length, ckpt_dir, starter_seq='The '):
        
        observed_seq = list(starter_seq)
        
        # restore model
        with tf.Session(graph=self.g) as sess:
            self.saver.restore(sess, 
                               tf.train.latest_checkpoint(ckpt_dir) )
            
            # run model using the starter sequence
            state = sess.run(self.initial_state)
            for char in observed_seq:
                x = np.zeros(shape=(1, 1))
                x[0, 0] = char2int[char]
                
                feed = {'tf_x:0': x, 
                        'tf_keepprob:0': 1.0,
                        self.initial_state: state}
                proba, state = sess.run(['probabilities:0', self.final_state], 
                                        feed_dict=feed)
            
            next_idx = get_top_char(proba, self.n_classes)
            observed_seq.append(int2char[next_idx])
            
            # run the model using the updated observed_seq
            for _ in range(output_length):
                x[0, 0] = next_idx
                feed = {'tf_x:0': x, 'tf_keepprob:0':1.0, 
                        self.initial_state: state}
                proba, state = sess.run(['probabilities:0', self.final_state], 
                                        feed_dict=feed)
                
                next_idx = get_top_char(proba, self.n_classes)
                observed_seq.append(int2char[next_idx])
            
        return "".join(observed_seq)
 

In [83]:
batch_size = 32
num_steps = 100

train_x, train_y = reshape_date(text_ints, batch_size, num_steps)

model = CharRnn(n_classes=len(char_set), batch_size=batch_size, n_steps=num_steps)

 << y_onehot >>  Tensor("one_hot_1:0", shape=(32, 100, 65), dtype=float32)
  << lstm_outputs  >>  Tensor("rnn/transpose:0", shape=(32, 100, 128), dtype=float32)
 << logits >>  Tensor("logits/BiasAdd:0", shape=(3200, 65), dtype=float32)


In [84]:
model.train(train_x, train_y, n_epochs=1000, ckpt_dir="C:/Users/p0ng5/Desktop/rnn_model/")

Epoch  0/1000, Iterations   20 | Train loss 3.23091
Epoch  0/1000, Iterations   40 | Train loss 3.14201
Epoch  1/1000, Iterations   60 | Train loss 3.15066
Epoch  1/1000, Iterations   80 | Train loss 3.14746
Epoch  1/1000, Iterations  100 | Train loss 3.09701
Epoch  2/1000, Iterations  120 | Train loss 3.09089
Epoch  2/1000, Iterations  140 | Train loss 3.02612
Epoch  3/1000, Iterations  160 | Train loss 3.00430
Epoch  3/1000, Iterations  180 | Train loss 2.98151
Epoch  3/1000, Iterations  200 | Train loss 2.89937
Epoch  4/1000, Iterations  220 | Train loss 2.89092
Epoch  4/1000, Iterations  240 | Train loss 2.83746
Epoch  5/1000, Iterations  260 | Train loss 2.83303
Epoch  5/1000, Iterations  280 | Train loss 2.84107
Epoch  5/1000, Iterations  300 | Train loss 2.76052
Epoch  6/1000, Iterations  320 | Train loss 2.79072
Epoch  6/1000, Iterations  340 | Train loss 2.75286
Epoch  7/1000, Iterations  360 | Train loss 2.74247
Epoch  7/1000, Iterations  380 | Train loss 2.72635
Epoch  7/100

Epoch 63/1000, Iterations 3180 | Train loss 2.48197
Epoch 63/1000, Iterations 3200 | Train loss 2.40939
Epoch 64/1000, Iterations 3220 | Train loss 2.44484
Epoch 64/1000, Iterations 3240 | Train loss 2.45490
Epoch 65/1000, Iterations 3260 | Train loss 2.45881
Epoch 65/1000, Iterations 3280 | Train loss 2.44700
Epoch 65/1000, Iterations 3300 | Train loss 2.39626
Epoch 66/1000, Iterations 3320 | Train loss 2.49094
Epoch 66/1000, Iterations 3340 | Train loss 2.43533
Epoch 67/1000, Iterations 3360 | Train loss 2.43199
Epoch 67/1000, Iterations 3380 | Train loss 2.48754
Epoch 67/1000, Iterations 3400 | Train loss 2.36570
Epoch 68/1000, Iterations 3420 | Train loss 2.42765
Epoch 68/1000, Iterations 3440 | Train loss 2.46714
Epoch 69/1000, Iterations 3460 | Train loss 2.45856
Epoch 69/1000, Iterations 3480 | Train loss 2.45396
Epoch 69/1000, Iterations 3500 | Train loss 2.38819
Epoch 70/1000, Iterations 3520 | Train loss 2.47010
Epoch 70/1000, Iterations 3540 | Train loss 2.42718
Epoch 71/100

Epoch 126/1000, Iterations 6320 | Train loss 2.36969
Epoch 126/1000, Iterations 6340 | Train loss 2.36614
Epoch 127/1000, Iterations 6360 | Train loss 2.39578
Epoch 127/1000, Iterations 6380 | Train loss 2.40138
Epoch 127/1000, Iterations 6400 | Train loss 2.32037
Epoch 128/1000, Iterations 6420 | Train loss 2.37142
Epoch 128/1000, Iterations 6440 | Train loss 2.36900
Epoch 129/1000, Iterations 6460 | Train loss 2.33817
Epoch 129/1000, Iterations 6480 | Train loss 2.36329
Epoch 129/1000, Iterations 6500 | Train loss 2.28036
Epoch 130/1000, Iterations 6520 | Train loss 2.37745
Epoch 130/1000, Iterations 6540 | Train loss 2.37558
Epoch 131/1000, Iterations 6560 | Train loss 2.38333
Epoch 131/1000, Iterations 6580 | Train loss 2.37784
Epoch 131/1000, Iterations 6600 | Train loss 2.28242
Epoch 132/1000, Iterations 6620 | Train loss 2.38174
Epoch 132/1000, Iterations 6640 | Train loss 2.38804
Epoch 133/1000, Iterations 6660 | Train loss 2.36865
Epoch 133/1000, Iterations 6680 | Train loss 2

Epoch 188/1000, Iterations 9420 | Train loss 2.33031
Epoch 188/1000, Iterations 9440 | Train loss 2.29204
Epoch 189/1000, Iterations 9460 | Train loss 2.35961
Epoch 189/1000, Iterations 9480 | Train loss 2.31846
Epoch 189/1000, Iterations 9500 | Train loss 2.28272
Epoch 190/1000, Iterations 9520 | Train loss 2.31879
Epoch 190/1000, Iterations 9540 | Train loss 2.30917
Epoch 191/1000, Iterations 9560 | Train loss 2.32545
Epoch 191/1000, Iterations 9580 | Train loss 2.34503
Epoch 191/1000, Iterations 9600 | Train loss 2.22636
Epoch 192/1000, Iterations 9620 | Train loss 2.32304
Epoch 192/1000, Iterations 9640 | Train loss 2.36196
Epoch 193/1000, Iterations 9660 | Train loss 2.30706
Epoch 193/1000, Iterations 9680 | Train loss 2.34526
Epoch 193/1000, Iterations 9700 | Train loss 2.20997
Epoch 194/1000, Iterations 9720 | Train loss 2.35365
Epoch 194/1000, Iterations 9740 | Train loss 2.30980
Epoch 195/1000, Iterations 9760 | Train loss 2.36541
Epoch 195/1000, Iterations 9780 | Train loss 2

Epoch 249/1000, Iterations 12480 | Train loss 2.33768
Epoch 249/1000, Iterations 12500 | Train loss 2.22374
Epoch 250/1000, Iterations 12520 | Train loss 2.28210
Epoch 250/1000, Iterations 12540 | Train loss 2.30507
Epoch 251/1000, Iterations 12560 | Train loss 2.28249
Epoch 251/1000, Iterations 12580 | Train loss 2.34767
Epoch 251/1000, Iterations 12600 | Train loss 2.20333
Epoch 252/1000, Iterations 12620 | Train loss 2.27592
Epoch 252/1000, Iterations 12640 | Train loss 2.32547
Epoch 253/1000, Iterations 12660 | Train loss 2.30033
Epoch 253/1000, Iterations 12680 | Train loss 2.28483
Epoch 253/1000, Iterations 12700 | Train loss 2.23877
Epoch 254/1000, Iterations 12720 | Train loss 2.29362
Epoch 254/1000, Iterations 12740 | Train loss 2.27772
Epoch 255/1000, Iterations 12760 | Train loss 2.31334
Epoch 255/1000, Iterations 12780 | Train loss 2.32882
Epoch 255/1000, Iterations 12800 | Train loss 2.17849
Epoch 256/1000, Iterations 12820 | Train loss 2.27429
Epoch 256/1000, Iterations 1

...

Epoch 735/1000, Iterations 36800 | Train loss 2.10190
Epoch 736/1000, Iterations 36820 | Train loss 2.22323
Epoch 736/1000, Iterations 36840 | Train loss 2.21782
Epoch 737/1000, Iterations 36860 | Train loss 2.16909
Epoch 737/1000, Iterations 36880 | Train loss 2.18800
Epoch 737/1000, Iterations 36900 | Train loss 2.07599
Epoch 738/1000, Iterations 36920 | Train loss 2.19153
Epoch 738/1000, Iterations 36940 | Train loss 2.23521
Epoch 739/1000, Iterations 36960 | Train loss 2.15335
Epoch 739/1000, Iterations 36980 | Train loss 2.18158
Epoch 739/1000, Iterations 37000 | Train loss 2.08713
Epoch 740/1000, Iterations 37020 | Train loss 2.16690
Epoch 740/1000, Iterations 37040 | Train loss 2.21799
Epoch 741/1000, Iterations 37060 | Train loss 2.18011
Epoch 741/1000, Iterations 37080 | Train loss 2.22052
Epoch 741/1000, Iterations 37100 | Train loss 2.14444
Epoch 742/1000, Iterations 37120 | Train loss 2.19015
Epoch 742/1000, Iterations 37140 | Train loss 2.22158
Epoch 743/1000, Iterations 3

Epoch 796/1000, Iterations 39840 | Train loss 2.22041
Epoch 797/1000, Iterations 39860 | Train loss 2.19672
Epoch 797/1000, Iterations 39880 | Train loss 2.19274
Epoch 797/1000, Iterations 39900 | Train loss 2.07805
Epoch 798/1000, Iterations 39920 | Train loss 2.20697
Epoch 798/1000, Iterations 39940 | Train loss 2.22026
Epoch 799/1000, Iterations 39960 | Train loss 2.21332
Epoch 799/1000, Iterations 39980 | Train loss 2.20356
Epoch 799/1000, Iterations 40000 | Train loss 2.09635
Epoch 800/1000, Iterations 40020 | Train loss 2.11743
Epoch 800/1000, Iterations 40040 | Train loss 2.16176
Epoch 801/1000, Iterations 40060 | Train loss 2.16513
Epoch 801/1000, Iterations 40080 | Train loss 2.17916
Epoch 801/1000, Iterations 40100 | Train loss 2.14910
Epoch 802/1000, Iterations 40120 | Train loss 2.18980
Epoch 802/1000, Iterations 40140 | Train loss 2.20838
Epoch 803/1000, Iterations 40160 | Train loss 2.22350
Epoch 803/1000, Iterations 40180 | Train loss 2.17034
Epoch 803/1000, Iterations 4

Epoch 857/1000, Iterations 42880 | Train loss 2.18304
Epoch 857/1000, Iterations 42900 | Train loss 2.09865
Epoch 858/1000, Iterations 42920 | Train loss 2.20073
Epoch 858/1000, Iterations 42940 | Train loss 2.17642
Epoch 859/1000, Iterations 42960 | Train loss 2.12409
Epoch 859/1000, Iterations 42980 | Train loss 2.18583
Epoch 859/1000, Iterations 43000 | Train loss 2.08967
Epoch 860/1000, Iterations 43020 | Train loss 2.13610
Epoch 860/1000, Iterations 43040 | Train loss 2.19738
Epoch 861/1000, Iterations 43060 | Train loss 2.14939
Epoch 861/1000, Iterations 43080 | Train loss 2.21732
Epoch 861/1000, Iterations 43100 | Train loss 2.07107
Epoch 862/1000, Iterations 43120 | Train loss 2.14134
Epoch 862/1000, Iterations 43140 | Train loss 2.19443
Epoch 863/1000, Iterations 43160 | Train loss 2.18781
Epoch 863/1000, Iterations 43180 | Train loss 2.14950
Epoch 863/1000, Iterations 43200 | Train loss 2.09575
Epoch 864/1000, Iterations 43220 | Train loss 2.17383
Epoch 864/1000, Iterations 4

Epoch 918/1000, Iterations 45920 | Train loss 2.16219
Epoch 918/1000, Iterations 45940 | Train loss 2.22379
Epoch 919/1000, Iterations 45960 | Train loss 2.21494
Epoch 919/1000, Iterations 45980 | Train loss 2.17346
Epoch 919/1000, Iterations 46000 | Train loss 2.09457
Epoch 920/1000, Iterations 46020 | Train loss 2.10048
Epoch 920/1000, Iterations 46040 | Train loss 2.13729
Epoch 921/1000, Iterations 46060 | Train loss 2.14604
Epoch 921/1000, Iterations 46080 | Train loss 2.17873
Epoch 921/1000, Iterations 46100 | Train loss 2.05427
Epoch 922/1000, Iterations 46120 | Train loss 2.11770
Epoch 922/1000, Iterations 46140 | Train loss 2.19604
Epoch 923/1000, Iterations 46160 | Train loss 2.13917
Epoch 923/1000, Iterations 46180 | Train loss 2.20017
Epoch 923/1000, Iterations 46200 | Train loss 2.04211
Epoch 924/1000, Iterations 46220 | Train loss 2.17413
Epoch 924/1000, Iterations 46240 | Train loss 2.23399
Epoch 925/1000, Iterations 46260 | Train loss 2.16899
Epoch 925/1000, Iterations 4

Epoch 979/1000, Iterations 48960 | Train loss 2.18501
Epoch 979/1000, Iterations 48980 | Train loss 2.18324
Epoch 979/1000, Iterations 49000 | Train loss 2.06587
Epoch 980/1000, Iterations 49020 | Train loss 2.19685
Epoch 980/1000, Iterations 49040 | Train loss 2.13610
Epoch 981/1000, Iterations 49060 | Train loss 2.16487
Epoch 981/1000, Iterations 49080 | Train loss 2.25273
Epoch 981/1000, Iterations 49100 | Train loss 2.07395
Epoch 982/1000, Iterations 49120 | Train loss 2.17334
Epoch 982/1000, Iterations 49140 | Train loss 2.20684
Epoch 983/1000, Iterations 49160 | Train loss 2.19345
Epoch 983/1000, Iterations 49180 | Train loss 2.20983
Epoch 983/1000, Iterations 49200 | Train loss 2.13374
Epoch 984/1000, Iterations 49220 | Train loss 2.11584
Epoch 984/1000, Iterations 49240 | Train loss 2.17281
Epoch 985/1000, Iterations 49260 | Train loss 2.15907
Epoch 985/1000, Iterations 49280 | Train loss 2.17706
Epoch 985/1000, Iterations 49300 | Train loss 2.10040
Epoch 986/1000, Iterations 4

In [85]:
train_x.shape, train_y.shape

((32, 5000), (32, 5000))

In [86]:
model_sampling = CharRnn(n_classes=len(char_set), sampling=True)

 << y_onehot >>  Tensor("one_hot_1:0", shape=(1, 1, 65), dtype=float32)
  << lstm_outputs  >>  Tensor("rnn/transpose:0", shape=(1, 1, 128), dtype=float32)
 << logits >>  Tensor("logits/BiasAdd:0", shape=(1, 65), dtype=float32)


In [87]:
sample_text = model_sampling.sample(output_length=500, ckpt_dir=model.ckpt_dir)

INFO:tensorflow:Restoring parameters from C:/Users/p0ng5/Desktop/rnn_model/char_model-50000.ckpt


In [88]:
print(sample_text)

The Snaretin hirs

   Ham. Ild mell, you will nom to himsence?
  Ham. The Maunerin withing that is io soued my
hine
Touee as the wirce of his like and the wine of his Seater,
As by his Sande, and whencestie of him to my.
Thet he do not that he shoued then world. I might better.
All wise the worting sinte and the mateer path,
Ah the euinke on my saces woued hime me alfene There
It heares
   Ham. Hill the Coopers, and shath my Lord

   Ham. Why thue my Lord, I haue brauerit my Lord

   Ham. Why see me 


In [73]:
print(sample_text)

The Procembin to the Project (oftentess. of aresendesters, for woredis indited tores,ofints aroted toothe
To tex that panders it tomsied forrance of this ancerier (of in it and fanmers of the prentecters/disites.  f our asents ones in if in our things and comsace of the pentrestiof
of mast.

Starse arices, and pookicinien arofe on copsed an on otsecitest to the sumbeste of areast parsocs if of mokestin on ar and thithen on on shosedanding.
         *  *]]  TO  UUEDORERADAAGENG-TI YoU IT NOPIIN DOMEST


In [68]:
print(sample_text)

The  it thile so do sente,
That tare sean anee in a torse at toue oo the soone soreet tie toores,
A   King. That a dildothis asariou,
Tee se a  ine the ener and homestie oon iole

   Ham. Ilat ou hathese to doue himes and orese antere, An  hat, one ar hild shere, If moue, on than in thane,
And shale the heare tit hes andot a thas arae a dang,
and wol  it in santise oo tie sith,
As  or whoue  onteet if mather
Ant this is tee e miserses and the mate,
Ard wo hou te har iot to mesthe at or heres to he me


In [69]:
model.best_model_path

'C:/Users/p0ng5/Desktop/rnn_model/char_model-2700.ckpt'